<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [3]:
import paths
from scripts import get_raw

In [4]:
get_raw.get_cases()

,city,confirmed,date,deaths,discarded,notes,notified,place_type,source_url,state,suspect,last_update
0,Almirante Tamandaré,0,2020-03-21,NaN,0.0,None,1.0,city,http://www.saude.pr.gov.br/arquivos/File/CORON...,PR,1.0,2020-03-21 15:57:57.188852
1,Altônia,0,2020-03-21,NaN,0.0,None,1.0,city,http://www.saude.pr.gov.br/arquivos/File/CORON...,PR,1.0,2020-03-21 15:57:57.188852
2,Alto Paraná,0,2020-03-21,NaN,3.0,None,3.0,city,http://www.saude.pr.gov.br/arquivos/File/CORON...,PR,0.0,2020-03-21 15:57:57.188852
3,Alvorada do Sul,0,2020-03-21,NaN,0.0,None,1.0,city,http://www.saude.pr.gov.br/arquivos/File/CORON...,PR,1.0,2020-03-21 15:57:57.188852
4,Ampére,0,2020-03-21,NaN,0.0,None,1.0,city,http://www.saude.pr.gov.br/arquivos/File/CORON...,PR,1.0,2020-03-21 15:57:57.188852
5,Apucarana,0,2020-03-21,NaN,1.0,None,3.0,city,http://www.saude.pr.gov.br/arquivos/File/CORON...,PR,2.0,2020-03-21 15:57:57.188852
6,Arapongas,0,2020-03-21,NaN,0.0,None,2.0,city,http://www.saude.pr.gov.br/arquivos/File/CORON...,PR,2.0,2020-03-21 15:57:57.188852
7,Araucária,0,2020-03-21,NaN,1.0,None,1.0,city,http://www.saude.pr.gov.br/arquivos/File/CORON...,PR,0.0,2020-03-21 15:57:57.188852
8,Assis Chateaubriand,0,2020-03-21,NaN,0.0,None,2.0,city,http://www.saude.pr.gov.br/arquivos/File/CORON...,PR,2.0,2020-03-21 15:57:57.188852
9,Bandeirantes,0,2020-03-21,NaN,0.0,None,2.0,city,http://www.saude.pr.gov.br/arquivos/File/CORON...,PR,2.0,2020-03-21 15:57:57.188852
